In [2]:
from pytorch_pretrained_bert import BertTokenizer
from os import listdir
wiki_data_path = "../Data/Fever/wiki-pages/"
training_data = "../Data/Fever/train.jsonl"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 428138.04B/s]


In [5]:
import json
data = []
with open(training_data, "r") as f:
    lines = f.read().split("\n")
    for line in lines:
        try:
            data.append(json.loads(line))
        except: 
            print("ignoring line:", line)

ignoring line: 


In [6]:
base_data = {}
ignored_lines = []
base_files = listdir(wiki_data_path)
for file in base_files:
    with open(wiki_data_path + file, "r") as f:
        lines = f.read().split("\n")
        for line in lines:
            try:
                _js = json.loads(line)
                base_data[_js["id"]] = _js
            except:
                ignored_lines.append(line)

In [7]:
import re
#from nltk.tokenize import sent_tokenize

def fix_text(text):
    text = text.replace(" -LRB- ", " (")
    text = text.replace(" -RRB- ", ") ")
    text = text.replace(" , ", ", ")
    text = text.replace(" . ", ". ")
    return text

processed_data = []

counter = 0
for line in data:
    counter = counter + 1
    print(counter, "/", len(data), end="\r")
    try:
        claim = line["claim"]
        if line["verifiable"] == "VERIFIABLE":
            _all_evidence = {}
            for evidence in line["evidence"]:
                for evid in evidence:
                    title = evid[2]
                    if (title is not None):
                        if (title not in _all_evidence):
                            _all_evidence[title] = []
                        if (evid[3] not in _all_evidence[title]):
                            _all_evidence[title].append(evid[3])
            #print(_all_evidence)
            line["evidences"] = _all_evidence
            claim = tokenizer.tokenize(claim)
            evidentiary = []
            non_evidentiary = []
            for source in _all_evidence:
                parts = fix_text(base_data[source]["lines"]).split("\n")
                for i in range(len(parts)):
                    part = parts[i]
                    passage_parts = part.split("\t")
                    if (len(passage_parts[1]) > 0):
                        if (i in _all_evidence[source]):
                            evidentiary.append(tokenizer.tokenize(passage_parts[1]))
                        else:
                            non_evidentiary.append(tokenizer.tokenize(passage_parts[1]))
            line["processed"] = {
                "claim":claim, 
                "evidentiary":evidentiary,
                "non_evidentiary":non_evidentiary
            }
            processed_data.append(line)
    except:
        print("Ignoring line:", line)
        

Ignoring line: {'id': 132873, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'José María Chacón surrendered the island of Trinidad in 1797, one year before his death.', 'evidence': [[[155336, 170035, 'José_María_Chacón', 0]]], 'evidences': {'José_María_Chacón': [0]}}
Ignoring line: {'id': 180425, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Régine Chassagne is Canadian.', 'evidence': [[[210899, 218545, 'Régine_Chassagne', 0]], [[210900, 218546, 'Régine_Chassagne', 0]]], 'evidences': {'Régine_Chassagne': [0]}}
Ignoring line: {'id': 189673, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Chris Pérez is a songwriter.', 'evidence': [[[221562, 227157, 'Chris_Pérez', 0]], [[221562, 227158, 'Chris_Pérez', 24]]], 'evidences': {'Chris_Pérez': [0, 24]}}
Ignoring line: {'id': 155901, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Beyoncé was given a music award.', 'evidence': [[[179964, 192503, 'Beyoncé', 16]], [[179964, 192504, 'Beyonce

Ignoring line: {'id': 34650, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'In Secret was an adaptation of a French novel.', 'evidence': [[[50830, 60521, 'In_Secret', 1], [50830, 60521, 'Thérèse_Raquin', 0]]], 'evidences': {'In_Secret': [1], 'Thérèse_Raquin': [0]}}
Ignoring line: {'id': 135798, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'Henri Poincare was an iPhone.', 'evidence': [[[158422, 173041, 'Henri_Poincaré', 0]], [[158422, 173042, 'Henri_Poincaré', 1]], [[158422, 173043, 'Henri_Poincaré', 4]], [[158422, 173044, 'Henri_Poincaré', 5]], [[158422, 173045, 'Henri_Poincaré', 6]], [[158422, 173046, 'Henri_Poincaré', 7]], [[158422, 173047, 'Henri_Poincaré', 10]], [[158422, 173048, 'Henri_Poincaré', 11]], [[158422, 173049, 'Henri_Poincaré', 12]], [[158422, 173050, 'Henri_Poincaré', 13]]], 'evidences': {'Henri_Poincaré': [0, 1, 4, 5, 6, 7, 10, 11, 12, 13]}}
Ignoring line: {'id': 14207, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Ci

Ignoring line: {'id': 44233, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'Beyoncé Knowles was listed by muffin as sixth place for Person of the Year.', 'evidence': [[[60633, 71053, 'Beyoncé', 20]]], 'evidences': {'Beyoncé': [20]}}
Ignoring line: {'id': 137095, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'The Uruguay national football team won competitions.', 'evidence': [[[159842, 174333, 'Uruguay_national_football_team', 4], [159842, 174333, '2011_Copa_América', 0]], [[159842, 174334, 'Uruguay_national_football_team', 5], [159842, 174334, 'Copa_América', 0]], [[159842, 174335, 'Uruguay_national_football_team', 6], [159842, 174335, 'FIFA_World_Cup', 0]], [[159842, 174336, 'Uruguay_national_football_team', 7], [159842, 174336, '1950_FIFA_World_Cup', 0]], [[159842, 174337, 'Uruguay_national_football_team', 10], [159842, 174337, 'Football_at_the_1924_Summer_Olympics', 0]], [[159842, 174338, 'Uruguay_national_football_team', 11], [159842, 174338, '1980_Mundia

Ignoring line: {'id': 96962, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Ingólfr Arnarson was the first permanent settler on Iceland, along with his wife and brother.', 'evidence': [[[114547, 128307, 'Iceland', 10], [114547, 128307, 'Ingólfr_Arnarson', 0]]], 'evidences': {'Iceland': [10], 'Ingólfr_Arnarson': [0]}}
Ignoring line: {'id': 165759, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Beyoncé Knowles is an actress.', 'evidence': [[[190604, 201600, 'Beyoncé', 0]]], 'evidences': {'Beyoncé': [0]}}
Ignoring line: {'id': 224652, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'William R. Tolbert Jr. was killed in a mutiny.', 'evidence': [[[267715, 264356, 'William_R._Tolbert_Jr.', 0], [267715, 264356, "Coup_d'état", 0]]], 'evidences': {'William_R._Tolbert_Jr.': [0], "Coup_d'état": [0]}}
Ignoring line: {'id': 184214, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Furia is penned by a human known for The Hills Have Eyes.', 'evidence'

Ignoring line: {'id': 192535, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Francois de Belleforest translated the works of Lodovico Guicciardini into French and he was French.', 'evidence': [[[225243, 229923, 'François_de_Belleforest', 0], [225243, 229923, 'François_de_Belleforest', 9]]], 'evidences': {'François_de_Belleforest': [0, 9]}}
Ignoring line: {'id': 166561, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'Christine Daaé is a real person.', 'evidence': [[[191564, 202396, 'Christine_Daaé', 0]]], 'evidences': {'Christine_Daaé': [0]}}
Ignoring line: {'id': 38889, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': "Beyoncé was listed among Time's 100 most influential people in the world.", 'evidence': [[[55116, 65154, 'Beyoncé', 19]]], 'evidences': {'Beyoncé': [19]}}
Ignoring line: {'id': 180451, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Régine Chassagne is a woman.', 'evidence': [[[213577, 220797, 'Régine_Chassagne', 1]]], 'e

...

Ignoring line: {'id': 42011, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'José María Chacón surrendered at least one island.', 'evidence': [[[58318, 68574, 'José_María_Chacón', 1]]], 'evidences': {'José_María_Chacón': [1]}}
Ignoring line: {'id': 122124, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Fahrenheit 451 was directed by a director.', 'evidence': [[[143414, 158532, 'Fahrenheit_451_-LRB-film-RRB-', 0], [143414, 158532, 'François_Truffaut', 0]]], 'evidences': {'Fahrenheit_451_-LRB-film-RRB-': [0], 'François_Truffaut': [0]}}
Ignoring line: {'id': 194968, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'Tré Cool is a lyricist, trumpeter, and composer.', 'evidence': [[[228611, 232776, 'Tré_Cool', 0]]], 'evidences': {'Tré_Cool': [0]}}
Ignoring line: {'id': 165766, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Beyoncé Knowles acts in Dreamgirls.', 'evidence': [[[190591, 201588, 'Beyoncé', 7]]], 'evidences': {'Beyoncé': [7]}}

Ignoring line: {'id': 166544, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Christine Daaé is character in The Phantom of the Opera by Gaston Leroux.', 'evidence': [[[191543, 202380, 'Christine_Daaé', 0]]], 'evidences': {'Christine_Daaé': [0]}}
Ignoring line: {'id': 181812, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'Beyoncé has only starred in Austin Powers.', 'evidence': [[[211372, 219090, 'Beyoncé', 7]]], 'evidences': {'Beyoncé': [7]}}
Ignoring line: {'id': 192394, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Francois de Belleforest translated the works of Saint Cyprian into French.', 'evidence': [[[225059, 229802, 'François_de_Belleforest', 9]]], 'evidences': {'François_de_Belleforest': [9]}}
Ignoring line: {'id': 189702, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Chris Pérez is a man.', 'evidence': [[[221680, 227249, 'Chris_Pérez', 4]], [[221680, 227250, 'Chris_Pérez', 6]], [[221680, 227251, 'Chris_Pérez', 11]], [[2

Ignoring line: {'id': 101223, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'Zlatan Ibrahimović refused to play for Barcelona.', 'evidence': [[[277155, 272121, 'Zlatan_Ibrahimović', 10]], [[278841, 273534, 'Zlatan_Ibrahimović', 10]], [[326884, 314266, 'Zlatan_Ibrahimović', 10]], [[326896, 314273, 'Zlatan_Ibrahimović', 10]]], 'evidences': {'Zlatan_Ibrahimović': [10]}}
Ignoring line: {'id': 86872, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Hugh Jackman starred in a movie musical.', 'evidence': [[[104007, 117512, 'Hugh_Jackman', 2], [104007, 117512, 'Les_Misérables_-LRB-2012_film-RRB-', 0]], [[104009, 117513, 'Hugh_Jackman', 2], [104009, 117513, 'Les_Misérables_-LRB-2012_film-RRB-', 0]]], 'evidences': {'Hugh_Jackman': [2], 'Les_Misérables_-LRB-2012_film-RRB-': [0]}}
Ignoring line: {'id': 165753, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Beyoncé Knowles starred in Obsessed.', 'evidence': [[[190584, 201583, 'Beyoncé', 7], [190584, 201583,

Ignoring line: {'id': 124834, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': "On top of the mountain Bonnet a L'Eveque is the location for the Citadelle Laferrière.", 'evidence': [[[146432, 161392, 'Citadelle_Laferrière', 0]]], 'evidences': {'Citadelle_Laferrière': [0]}}
Ignoring line: {'id': 107835, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'The National Council for Peace and Order was formed by the people.', 'evidence': [[[126566, 140964, 'National_Council_for_Peace_and_Order', 0], [126566, 140964, "2014_Thai_coup_d'état", 1]]], 'evidences': {'National_Council_for_Peace_and_Order': [0], "2014_Thai_coup_d'état": [1]}}
Ignoring line: {'id': 36734, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'The Citadelle Laferrière is only a small fortress that sits on the bottom of a mountain.', 'evidence': [[[52923, 62851, 'Citadelle_Laferrière', 0]], [[52923, 62852, 'Citadelle_Laferrière', 2]]], 'evidences': {'Citadelle_Laferrière': [0, 2]}}
Ignoring li

Ignoring line: {'id': 44197, 'verifiable': 'VERIFIABLE', 'label': 'REFUTES', 'claim': 'José María Chacón was responsible for signing the yearbook in 1783.', 'evidence': [[[60535, 70965, 'José_María_Chacón', 1]]], 'evidences': {'José_María_Chacón': [1]}}
Ignoring line: {'id': 223022, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'The Thénardiers are in a book.', 'evidence': [[[265672, 262862, 'Thénardiers', 0]], [[265672, 262863, 'Thénardiers', 4]], [[265672, 262864, 'Thénardiers', 6], [265672, 262864, 'Jean_Valjean', 0]], [[265672, 262865, 'Thénardiers', 7], [265672, 262865, 'Javert', 0]], [[265672, 262866, 'Thénardiers', 8]]], 'evidences': {'Thénardiers': [0, 4, 6, 7, 8], 'Jean_Valjean': [0], 'Javert': [0]}}
Ignoring line: {'id': 134480, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Ricky Martin is a singer.', 'evidence': [[[157031, 171589, 'Ricky_Martin', 0]], [[157031, 171590, 'Ricky_Martin', 10], [157031, 171590, 'Ricky_Martin_-LRB-1999_album-RR

In [8]:
print(len(processed_data))

108771


In [9]:
import pickle
pickle.dump(processed_data, open("../fever_processed.pickle", "wb"))